In [1]:
import math

###############################################################################
# 1) Basic Functions: Strength Reduction, Steel Strain, Beta_1
###############################################################################

def calculate_strength_reduction_factor(actual_epsilon, epsilon_ty, section_type):
    if not isinstance(actual_epsilon, (int, float)) or not isinstance(epsilon_ty, (int, float)):
        raise ValueError("actual_epsilon and epsilon_ty must be numbers.")
    if section_type not in ["spiral", "others"]:
        raise ValueError("section_type must be either 'spiral' or 'others'.")

    epsilon_t = 0.003 + epsilon_ty
    if actual_epsilon <= epsilon_ty:
        # compression-controlled
        phi = 0.75 if section_type == "spiral" else 0.65
    elif actual_epsilon < epsilon_t:
        # transition
        factor = (actual_epsilon - epsilon_ty) / 0.003
        if section_type == "spiral":
            phi = 0.75 + 0.15 * factor
        else:
            phi = 0.65 + 0.25 * factor
    else:
        # tension-controlled
        phi = 0.90
    return phi

def steel_yield_strain(fy, es):
    epsilon_s = fy / es
    return {
        "value": epsilon_s,
        "report": f"yield steel strain, epsilon_s = {epsilon_s:.6f}"
    }

def calculate_beta_one(fc):
    if fc < 17:
        raise ValueError("fc < 17 MPa is out of scope for this formula.")
    if 17 <= fc <= 28:
        beta_one = 0.85
        rep = "beta_1 = 0.85"
    elif 28 < fc < 55:
        beta_one = 0.85 - ((0.05 * (fc - 28)) / 7.0)
        rep = (f"beta_1 = 0.85 - (0.05*(fc - 28))/7\n"
               f"        = 0.85 - (0.05*({fc:.2f}-28))/7\n"
               f"        = {beta_one:.2f}")
    else:  # fc >= 55
        beta_one = 0.65
        rep = "beta_1 = 0.65"
    return {"value": beta_one, "report": rep}


###############################################################################
# 2) Rebar Processing & Partitioning
###############################################################################

def process_rebar_data(rebar_data):
    processed = []
    lines = []
    lines.append("rebar data (d, diam, num, as):")
    for rebar in rebar_data:
        area = math.pi * (rebar["diam"]/2)**2 * rebar["num"]
        item = dict(rebar)
        item["as"] = area
        processed.append(item)
        lines.append(f"  d={rebar['d']:7.2f}, diam={rebar['diam']:5.2f}, num={rebar['num']:2d}, as={area:10.2f}")
    report = "\n".join(lines)
    return {"value": processed, "report": report}

def filter_rebar_data(rebars, neutral_axis):
    compression = []
    tension = []
    as_top = 0.0
    as_bot = 0.0

    for bar in rebars:
        if bar["d"] < neutral_axis:
            compression.append(bar)
            as_top += bar["as"]
        else:
            tension.append(bar)
            as_bot += bar["as"]

    dc = min((b["d"] for b in compression), default=None) if compression else None
    dt = max((b["d"] for b in tension), default=None) if tension else None
    eff_dc = (sum(b["d"] for b in compression)/len(compression)) if compression else None
    eff_dt = (sum(b["d"] for b in tension)/len(tension)) if tension else None

    return {
        "compression": compression,
        "tension": tension,
        "dc": dc,
        "dt": dt,
        "eff_dc": eff_dc,
        "eff_dt": eff_dt,
        "as_top": as_top,
        "as_bot": as_bot
    }

def generate_compression_tension_tables(compression_data, tension_data):
    comp_lines = ["compression rebar (d, diam, num, as):"]
    for bar in compression_data:
        comp_lines.append(f"  d={bar['d']:7.2f}, diam={bar['diam']:5.2f}, num={bar['num']:2d}, as={bar['as']:10.2f}")

    tens_lines = ["tension rebar (d, diam, num, as):"]
    for bar in tension_data:
        tens_lines.append(f"  d={bar['d']:7.2f}, diam={bar['diam']:5.2f}, num={bar['num']:2d}, as={bar['as']:10.2f}")

    return {
        "compression": "\n".join(comp_lines),
        "tension": "\n".join(tens_lines)
    }


###############################################################################
# 3) The EXACT solver.js iteration (two-phase approach) for neutral axis
###############################################################################

def calculate_rebar_forces_solverjs_style(
    rebar_list, beam_height, beam_width,
    fc, fy, es, ecu, steel_strain, beta_one
):
    
    dt = beam_height
    current_x = dt
    iter_num = 50
    iter_step = dt / iter_num
    update_step = True
    alpha_2 = 0.85

    x_output = None
    solution_found = False

    iteration_data = []
    i = 0
    pass_count = 1

    # final forces
    fcc_final = 0.0
    fcsi_final = 0.0
    fsi_final = 0.0
    ratio_final = None

    while i < iter_num:
        if i == 0:
            x = current_x
        else:
            x = current_x - iter_step
            current_x = x

        # compute compression & tension rebar
        fcsi_rebar = 0.0
        fsi_rebar = 0.0
        for bar in rebar_list:
            d_i = bar["d"]
            as_i = bar["as"]
            if d_i < x:
                # compression zone
                esci = ((x - d_i)/x)*ecu
                f_sci = min(esci*es, fy)
                fi = as_i*f_sci
                fcsi_rebar += fi
            else:
                # tension zone
                esti = ((d_i - x)/x)*ecu
                f_sti = min(esti*es, fy)
                fi = as_i*f_sti
                fsi_rebar += fi

        # concrete block
        a_comp = beta_one * x * beam_width
        f_concrete = alpha_2 * fc * a_comp
        fci = f_concrete + fcsi_rebar
        fsi = fsi_rebar
        ratio = fci/fsi if fsi != 0 else float('inf')

        iteration_data.append({
            "pass": pass_count,
            "iteration": i+1,
            "x": x,
            "fc_concrete_kN": f_concrete/1000.0,
            "fc_rebar_kN": fcsi_rebar/1000.0,
            "fs_rebar_kN": fsi_rebar/1000.0,
            "fci_kN": (f_concrete + fcsi_rebar)/1000.0,
            "ratio": ratio
        })

        if not update_step:
            if fci <= fsi:
                x_output = x
                fcc_final = f_concrete
                fcsi_final = fcsi_rebar
                fsi_final = fsi_rebar
                ratio_final = ratio
                solution_found = True
                break
        else:
            if fci <= fsi:
                update_step = False
                # store final
                x_output = x
                fcc_final = f_concrete
                fcsi_final = fcsi_rebar
                fsi_final = fsi_rebar
                ratio_final = ratio
                solution_found = True

                # revert x by adding step
                current_x += iter_step
                # refine step
                iter_step = iter_step / iter_num
                pass_count += 1
                i = -1  # so next loop iteration will be i=0
        i += 1

    return {
        "iteration_data": iteration_data,
        "neutral_axis": x_output,
        "fc_concrete": fcc_final,
        "fc_rebar": fcsi_final,
        "fs_rebar": fsi_final,
        "ratio": ratio_final
    }


###############################################################################
# 4) Main "generate_beam_report" that uses the solverjs-style iteration
###############################################################################

def generate_beam_report(rebar_data, fc, fy, beam_height, beam_width, ds, clear_cover):
    
    # Validate
    if not isinstance(rebar_data, list) or len(rebar_data) == 0:
        return {"report": "", "mn": None, "mu": None, "error": "invalid rebar_data"}
    if fc <= 0 or fy <= 0 or beam_height <= 0 or beam_width <= 0 or ds <= 0 or clear_cover < 0:
        return {"report": "", "mn": None, "mu": None, "error": "invalid input parameters"}

    # Start building a plain-text report
    lines = []
    lines.append("----- BEAM DESIGN REPORT -----")

    # Concrete strain
    ecu = 0.003
    lines.append(f"concrete strain, epsilon_c = {ecu:.4f}")

    # Steel yield strain
    es_mod = 200000.0  # MPa
    sstrain = steel_yield_strain(fy, es_mod)
    epsilon_s = sstrain["value"]
    lines.append(sstrain["report"])

    # beta_1
    b1 = calculate_beta_one(fc)
    beta_one = b1["value"]
    lines.append(b1["report"])

    # rebar data
    processed = process_rebar_data(rebar_data)
    rebar_list = processed["value"]
    lines.append(processed["report"])

    # iteration for neutral axis
    forces = calculate_rebar_forces_solverjs_style(
        rebar_list, beam_height, beam_width,
        fc, fy, es_mod, ecu, epsilon_s, beta_one
    )

    # Print iteration table
    lines.append("\niteration results (matching solver.js approach):")
    lines.append(" pass  iter    c(mm)   fc_conc(kN)  fc_rebar(kN)  fs_rebar(kN)  total_fc(kN)  ratio")
    for row in forces["iteration_data"]:
        lines.append(
            f" {row['pass']:4d}  {row['iteration']:4d}  {row['x']:8.2f}"
            f"  {row['fc_concrete_kN']:11.2f}  {row['fc_rebar_kN']:11.2f}  {row['fs_rebar_kN']:11.2f}"
            f"  {row['fci_kN']:11.2f}  {row['ratio']:.3f}"
        )

    c_val = forces["neutral_axis"]
    if c_val is None:
        lines.append("\n*** could not find balanced neutral axis ***")
        return {"report": "\n".join(lines), "mn": None, "mu": None, "error": "no solution found"}

    lines.append(f"\nfinal neutral axis c = {c_val:.2f} mm")

    # partition rebar
    parted = filter_rebar_data(rebar_list, c_val)
    tables = generate_compression_tension_tables(parted["compression"], parted["tension"])
    lines.append("\n" + tables["compression"])
    lines.append("\n" + tables["tension"])

    a_val = beta_one * c_val
    lines.append(f"\ndepth of rectangular block, a = beta_1*c = {beta_one:.2f} * {c_val:.2f} = {a_val:.2f} mm")

    # actual tension strain
    dt_eff = parted["dt"] if parted["dt"] is not None else beam_height
    actual_epsilon_t = ((dt_eff - c_val) / c_val) * ecu
    lines.append(f"actual tension strain, epsilon_t = {actual_epsilon_t:.6f}")

    # strength reduction factor
    phi_val = calculate_strength_reduction_factor(actual_epsilon_t, epsilon_s, "others")
    lines.append(f"strength reduction factor, phi = {phi_val:.2f}")

    f_concrete_n = forces["fc_concrete"]   # N
    f_compsteel_n = forces["fc_rebar"]     # N

    dt = parted["dt"]
    dc = parted["dc"]
    if dt is None:
        dt = beam_height  # fallback if no tension rebar
    if dc is None:
        mn_nmm = f_concrete_n * (dt - a_val/2.0)
        lines.append("\n(no compression bars): M_n = Fc*(d - a/2)")
    else:
        mn_nmm = (f_concrete_n * (dt - a_val/2.0)) + (f_compsteel_n * (dt - dc))
        lines.append("\n(with compression bars): M_n = Fc*(dt - a/2) + Fsc*(dt - dc)")

    mn = mn_nmm / 1e6
    mu = phi_val * mn

    lines.append(f"nominal moment capacity, M_n = {mn:.2f} kN·m")
    lines.append(f"ultimate moment capacity, M_u = phi*M_n = {phi_val:.2f} * {mn:.2f} = {mu:.2f} kN·m")

    full_report = "\n".join(lines)
    return {"report": full_report, "mn": mn, "mu": mu, "error": None}


###############################################################################
# 5) CALCULATE FOR ULTIMATE MOMENT OF THE BEAM
###############################################################################

def beamsolver():
    rebars = [
        {'d': 58,  'diam': 16, 'num': 2},  # top (compression) bars
        {'d': 342, 'diam': 16, 'num': 3},  # bottom (tension) bars
    ]

    result = generate_beam_report(
        rebar_data=rebars,
        fc=20.68,
        fy=414.0,
        beam_height=400.0,
        beam_width=200.0,
        ds=10.0,
        clear_cover=40.0
    )

    if result["error"]:
        print("ERROR:", result["error"])
    else:
        print(result["report"])
        print(f"\nNominal Moment (M_n) = {result['mn']:.2f} kN·m")
        print(f"Ultimate Moment (M_u) = {result['mu']:.2f} kN·m")

# Uncomment the line below to run the demo in Jupyter:
beamsolver()

----- BEAM DESIGN REPORT -----
concrete strain, epsilon_c = 0.0030
yield steel strain, epsilon_s = 0.002070
beta_1 = 0.85
rebar data (d, diam, num, as):
  d=  58.00, diam=16.00, num= 2, as=    402.12
  d= 342.00, diam=16.00, num= 3, as=    603.19

iteration results (matching solver.js approach):
 pass  iter    c(mm)   fc_conc(kN)  fc_rebar(kN)  fs_rebar(kN)  total_fc(kN)  ratio
    1     1    400.00      1195.30       218.96         0.00      1414.26  inf
    1     2    392.00      1171.40       212.64         0.00      1384.04  inf
    1     3    384.00      1147.49       206.06         0.00      1353.56  inf
    1     4    376.00      1123.59       199.21         0.00      1322.79  inf
    1     5    368.00      1099.68       192.05         0.00      1291.73  inf
    1     6    360.00      1075.77       184.57         0.00      1260.35  inf
    1     7    352.00      1051.87       176.76         0.00      1228.63  inf
    1     8    344.00      1027.96       168.58         0.00      